Here introduces the related functions for creating HEA slab models.

# Creating a single HEA slab structur

The creation of a single HEA slab requires the sequential use of Function **substitute_in_probability** and Function **average_lattice**. 
- **substitute_in_probability** replaces a pure-element initial surface with an HEA structure through element substitution.
- **average_lattice** then modifies the lattice parameters based on the substituted configuration to introduce lattice distortion effects.

Let us import them at first.

In [8]:
from heaict.hea.slab import substitute_in_probability, average_lattice, save_structure
from pymatgen.core import Structure

## substitute_in_probability
- create a surface composed of pure elements.
- specify in a dictionary the element to be replaced, the substituting elements, and their respective fraction ratios.

Here we initially create a surface composed entirely of Cu elements, and then replace Cu with Ni, Pd, Cu, Fe, and Ru in equal proportions.

In [4]:
origin_Cu_slab = Structure.from_file(f'Data/Example/HEA_slab/origin_Cu_slab.vasp')

In [6]:
HEA_slab = substitute_in_probability(
    struc=origin_Cu_slab,
    sub_pro={'Cu': {'Ni': 0.2, 'Pd': 0.2, 'Cu': 0.2, 'Fe': 0.2, 'Ru': 0.2}},
    method='probability'
)

After the replacement, the atomic quantities of each element are as follows.   

Since the replacement is performed based on probability, the number of atoms for each element differs. You may also choose to conduct the replacement using a proportional method instead.

In [11]:
ele_num = [0, 0, 0, 0, 0]
for ele in HEA_slab:
    if   ele.specie.symbol == 'Ni': ele_num[0] += 1
    elif ele.specie.symbol == 'Pd': ele_num[1] += 1
    elif ele.specie.symbol == 'Cu': ele_num[2] += 1
    elif ele.specie.symbol == 'Fe': ele_num[3] += 1
    elif ele.specie.symbol == 'Ru': ele_num[4] += 1
ele_num

[6, 7, 10, 7, 6]

## average_lattice
Function **average_lattice** is responsible for scaling the lattice constants of the slab to account for lattice distortion effects.   

The inputs required for this function are:
- The HEA slab structure with elements already replaced.
- The lattice constants corresponding to each element's structure. Here, we use the lattice constants of each element in the FCC structure.
- The lattice constant of the initial slab model. Our initial slab model is constructed based on a bulk structure with a lattice constant of 3.63.  

During the lattice scaling process, two steps are performed:
- Scale the lattice of the slab structure in all three directions based on the average lattice constant of all atoms in the slab.
- Scale the horizontal lattice of the slab structure based on the average lattice constant of the outermost atoms. This step can be toggled using parameter **surface_adjusted**.

In [13]:
origin_lattice = 3.63

data_lat = {'Mo':4.0039480569830976,
            'Mn':3.5016593728624845,
            'Fe':3.6363044593942768,
            'Ru':3.8095553061967871,
            'Co':3.5164448514901356,
            'Ni':3.5170623625640007,
            'Pd':3.9411525946094144,
            'Cu':3.6349893313983990,}

HEA_slab_lattice_adjusted = average_lattice(
    struc=HEA_slab,
    data_lat=data_lat,
    origin_lattice=origin_lattice,
    vacuum=15,
    surface_adjusted=True
)

Comparing the lattice parameters of the slab before and after scaling reveals a noticeable difference.

In [15]:
HEA_slab.lattice.matrix

array([[ 7.700393,  0.      ,  0.      ],
       [-3.850196,  6.668736,  0.      ],
       [ 0.      ,  0.      , 21.287344]])

In [16]:
HEA_slab_lattice_adjusted.lattice.matrix

array([[ 7.81337233e+00,  0.00000000e+00,  4.78431071e-16],
       [-3.90668566e+00,  6.76657897e+00,  4.78431058e-16],
       [ 0.00000000e+00,  0.00000000e+00,  2.14158910e+01]])

Finally, save the created HEA slab model.

In [18]:
save_structure(HEA_slab_lattice_adjusted, f'Data/Example/HEA_slab/HEA_slab_example.vasp')

# Batch creation of HEA slabs. 
We have integrated the entire process described above into Function **slab_pure_to_hea** to enable random batch creation of HEA slab structures.

Based on the original setup, batch creation can be performed by additionally specifying all candidate elements, the number of elements in each slab structure, and the number of slab structures to be generated.  

Here, we use a total of 8 elements to create HEA slab structures. Each slab can contain either 4 or 5 elements, and a total of 10 slabs will be created for demonstration purposes.

After the creation process is complete, a data table will also be generated to display the elemental composition (fractions) of each slab structure.

In [19]:
from heaict.hea.slab import slab_pure_to_hea

In [21]:
df = slab_pure_to_hea(
    origin_slab=origin_Cu_slab,
    origin_ele='Cu',
    eleAlat=data_lat,
    num_ele=[4, 5],
    num_hea=10,
    save_path=f'Data/Example/HEA_slab/',
    vacuum=15,
    surface_adjusted=True,
    save_by_index=True,
)

In [22]:
df

,index,HEA
0,1,Mo-16_Mn-2_Co-18_Ru-13_Pd-51
1,2,Ru-53_Co-12_Fe-26_Ni-9
2,3,Fe-22_Mo-19_Co-0_Cu-20_Ru-39
3,4,Mo-5_Fe-5_Mn-40_Pd-50
4,5,Ni-7_Pd-20_Co-1_Ru-18_Mo-54
5,6,Ni-26_Mn-27_Pd-23_Ru-24
6,7,Cu-33_Mo-12_Co-1_Ni-12_Pd-42
7,8,Cu-31_Fe-31_Ni-18_Mn-3_Co-17
8,9,Ru-38_Pd-4_Mo-56_Fe-2
9,10,Mo-24_Co-11_Pd-14_Ni-26_Fe-25
